## Analysis of the Results:
This notebook includes the following analyses:
* Overall changes in responses before exposure to articles and after exposure to articles
  * Overall difference in responses between left and right biased articles exposure
  * Overall differences in responses by question
  * Differences in responses by political groups
  * Differences in responses by article bias: which bias causes the greater changes and in which direction?
  * Differences in responses by article bias by political group
  * Differences in responses by article bias, by political group, and by question
* Radicalization analysis:
  * Reinforcement of right wing opinions if right wing users are exposed to right wing biased articles
  * Reinforcement of left wing opinions if left wing are exposed to left wing biased articles
* LLM to persona alignment before vs after exposure to articles: (is the llm more consistent with the actual person's responses after it was exposed to a. right biased article b. left biased articles.)
* Significance test: is the change caused by the articles?
  

#### Data Extraction and Processing

In [188]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import re


In [189]:
# Modified version of load_json_data to print more details about malformation
def load_json_data(filepath):
    """Loads JSON data from the provided file path and attempts to fix malformed sections."""
    try:
        with open(filepath, "r", encoding="utf-8") as f:
            content = f.read()

        # First attempt to load the JSON directly
        try:
            return json.loads(content)  # If well-formed, this will work
        except json.JSONDecodeError as e:
            print(f"Malformed JSON detected: {str(e)}")
            print("Attempting to fix...")

            # Fix common malformation: misplaced brackets (e.g., '][')
            content_fixed = content.replace("][", "],[")

            # Attempt to load the fixed content
            try:
                return json.loads(f"[{content_fixed}]")  # Wrap content in square brackets
            except json.JSONDecodeError as e:
                print(f"Error: Unable to decode JSON after attempt to fix. {str(e)}")
                return None

    except FileNotFoundError:
        print(f"Error: File {filepath} not found.")
        return None
    except json.JSONDecodeError as e:
        print(f"Error: Failed to decode JSON from {filepath}. {str(e)}")
        return None


In [190]:
def flatten_after_responses(response_data):
    """
    Flattens the nested structure of the after responses data.
    Converts the list of questions for each user into individual rows.
    """
    flattened_data = []
    
    for user_responses in response_data:
        for entry in user_responses:
            user_id = entry['user_id']
            question_code = entry['question_code']
            bias = entry['bias']
            selected_option = entry['response'].get('selected_option')

            # Append a flattened version of the entry
            flattened_data.append({
                'user_id': user_id,
                'question_code': question_code,
                'bias': bias,
                'selected_option': selected_option,
                'question': entry.get('question')
            })
    
    return flattened_data


#### Paths to Data

In [191]:
before_responses_path = "../data/processed/before_responses.json"
after_responses_path = "../data/processed/after_responses.json"
persona_prompts_path = '../data/processed/persona_prompts.json'
question_codes_path = '../data/raw/question_codes.json'





In [192]:
def load_question_code_mapping(question_codes_path):
    """Load and filter the question-to-code mapping from the question_codes.json file."""
    question_codes_data = load_json_data(question_codes_path)
    
    # List of valid question codes relevant to the analysis
    valid_question_codes = ['F1A10_1', 'F2A6', 'F2A7', 'F2A9', 'F3A3_1', 'F3A6_1', 'F3A7_1', 'F3A8_1']
    
    # Filter only questions that are in the valid_question_codes list
    filtered_data = [entry for entry in question_codes_data if entry['code'] in valid_question_codes]

    # Create a mapping for each question from text to numerical codes
    question_code_mapping = {}
    for entry in filtered_data:
        question = entry['question']
        code = entry['code']
        options = entry['options']
        
        # Reverse the options dictionary to map response text to numerical values
        reversed_options = {v: k for k, v in options.items()}
        question_code_mapping[question] = {'code': code, 'options': reversed_options}

    return question_code_mapping


In [193]:
def create_after_responses_dataframe(flattened_data):
    """
    Converts the flattened after responses into a structured DataFrame.
    """
    df = pd.DataFrame(flattened_data)
    
    # Set a multi-index using user_id, question_code, and bias
    df.set_index(['user_id', 'question_code', 'bias'], inplace=True)
    
    return df


In [194]:
def extract_actual_question(full_prompt):
    """Extracts the actual question from the full prompt in the 'question' field and cleans it."""
    
    # Use regex to find text between **Question**: and **Options**:
    match = re.search(r'\*\*Question\*\*[:\s]*(.*?)\*\*Options\*\*', full_prompt, re.DOTALL)
    
    if match:
        question = match.group(1).strip()  # Extracted question
        # Clean the question: remove extra spaces and newlines
        question_cleaned = re.sub(r'\s+', ' ', question).strip()
        return question_cleaned
    
    # If no structured question is found, return the full prompt (fallback)
    return full_prompt.strip()


In [195]:

def create_clean_dataframe_with_codes(response_data, question_code_mapping, has_bias=False):
    """
    Converts a list of response data dictionaries into a structured DataFrame with question codes.
    Handles optional fields like bias if present in the data (for after_responses).
    Parameters:
    - response_data: List of dictionaries containing the response data.
    - question_code_mapping: A dictionary that maps question texts to their corresponding codes.
    - has_bias: Boolean flag to indicate if the data includes bias (for after_responses).
    """
    cleaned_data = []
    
    for entry in response_data:
        user_id = entry['user_id']
        full_prompt = entry['question']  # Full prompt in the before_responses
        selected_option = entry['response']['selected_option']
        bias = entry.get('bias', None) if has_bias else None

        # Step 1: Extract the actual question from the full prompt
        question = extract_actual_question(full_prompt)
        
        if question is None:
            print(f"Warning: Could not extract question from prompt for user {user_id}. Skipping entry.")
            continue

        # Step 2: Look up the question_code from the extracted question text
        question_code = question_code_mapping.get(question, {}).get('code')
        
        if question_code is None:
            print(f"Warning: No question_code found for question '{question}' for user {user_id}. Skipping entry.")
            continue

        # Build the cleaned entry
        cleaned_entry = {
            'user_id': user_id,
            'question_code': question_code,
            'bias': bias,
            'selected_option': selected_option,
            'question': question
        }
        
        cleaned_data.append(cleaned_entry)
    
    # Convert cleaned data into a DataFrame and use MultiIndex if bias is included
    df = pd.DataFrame(cleaned_data)
    if has_bias:
        df.set_index(['user_id', 'question_code', 'bias'], inplace=True)
    else:
        df.set_index(['user_id', 'question_code'], inplace=True)
    
    return df


In [196]:
def map_responses_to_numeric(df, question_code_mapping):
    """
    Maps the string-based responses to their corresponding numerical values using the question code mapping.
    """
    def map_response(row):
        question_text = row['question']  # Use the cleaned question column
        selected_option = row['selected_option']  # Directly access the selected option column
        
        # Find the mapping for the current question text
        if question_text in question_code_mapping:
            mapping = question_code_mapping[question_text]['options']
            # Return the numerical code for the response
            return mapping.get(selected_option, None)  # Return None if no match found
        return None
    
    # Apply the mapping function to each row
    df['numeric_response'] = df.apply(map_response, axis=1)
    return df


In [197]:
def extract_political_stance(persona_prompt):
    """Extracts the political stance from the persona prompt text."""
    match = re.search(r"\*\*Your Own Political Position\*\*:\s*You consider your political position to be\s*'(.*?)'\s*on the political scale", persona_prompt)
    
    if match:
        return match.group(1).strip()  # Extract the political position (e.g., 'Extreme Left')
    return None


# Load the persona prompts
def load_persona_prompts(filepath):
    """Loads the persona prompts data and creates a mapping of user_id to political stance."""
    persona_data = load_json_data(filepath)  # Reusing load_json_data to load the file
    if persona_data is None:
        return {}

    # Create a mapping of user_id to political stance
    persona_mapping = {}
    for entry in persona_data:
        user_id = entry['user_id']
        political_stance = extract_political_stance(entry['persona_prompt'])
        persona_mapping[user_id] = political_stance
    
    return persona_mapping

def add_political_stance_to_df(df, persona_mapping):
    """Adds political stance column to the given DataFrame based on user_id, and combines Far Left and Extreme Left."""
    # Map political stance from persona_mapping to the DataFrame
    df['political_stance'] = df.index.get_level_values('user_id').map(persona_mapping)

    # Merge Far Left and Extreme Left into a single group: Extreme Left
    df['political_stance'] = df['political_stance'].replace({'Far Left': 'Extreme Left'})

    # Debug: Check for missing political stance mappings
    missing_stance_df = df[df['political_stance'].isna()]
    if not missing_stance_df.empty:
        print("These user_ids have no political stance mapped:")
        print(missing_stance_df.index.get_level_values('user_id').unique())

    return df



In [198]:

# Load the question code mapping for both before and after responses
question_code_mapping = load_question_code_mapping(question_codes_path)
persona_mapping = load_persona_prompts(persona_prompts_path)

## Process the BEFORE data

before_responses = load_json_data(before_responses_path)
before_responses_df = create_clean_dataframe_with_codes(before_responses, question_code_mapping)
before_responses_df = map_responses_to_numeric(before_responses_df, question_code_mapping)
before_responses_df = add_political_stance_to_df(before_responses_df, persona_mapping)

## Process the AFTER data

after_responses = load_json_data(after_responses_path)
flattened_after_responses = flatten_after_responses(after_responses)
after_responses_df = create_after_responses_dataframe(flattened_after_responses)
after_responses_df = map_responses_to_numeric(after_responses_df, question_code_mapping)
after_responses_df = add_political_stance_to_df(after_responses_df, persona_mapping)

## Next Steps: Analyze the data

# Print the first few rows of both DataFrames to verify the result
#print("Before Responses DataFrame:")
#print(before_responses_df.head())

#print("\nAfter Responses DataFrame:")
#print(after_responses_df.head())


Malformed JSON detected: Extra data: line 178 column 2 (char 5656)
Attempting to fix...


#### Dataframe for the following steps

In [199]:
# Ensure the numeric_response columns are in numeric format before merging
def ensure_numeric_format(df, column):
    df[column] = pd.to_numeric(df[column], errors='coerce')
    return df

before_responses_df = ensure_numeric_format(before_responses_df, 'numeric_response')
after_responses_df = ensure_numeric_format(after_responses_df, 'numeric_response')

# Merge the before and after responses DataFrames on user_id and question_code
def merge_before_after_responses(before_df, after_df):
    # Reset the index to bring 'user_id' and 'question_code' back as columns
    before_df = before_df.reset_index()
    after_df = after_df.reset_index()

    merged_df = pd.merge(
        before_df[['user_id', 'question_code', 'numeric_response', 'political_stance']],
        after_df[['user_id', 'question_code', 'numeric_response', 'bias', 'political_stance']],
        on=['user_id', 'question_code', 'political_stance'],  # Merge on user_id, question_code, and political_stance
        suffixes=('_before', '_after')
    )

    # Calculate the change in response (after - before)
    merged_df['response_change'] = merged_df['numeric_response_after'] - merged_df['numeric_response_before']

    return merged_df

# Merge the DataFrames
merged_responses_df = merge_before_after_responses(before_responses_df, after_responses_df)

# Display the merged DataFrame
print(merged_responses_df.head())


      user_id question_code  numeric_response_before political_stance  \
0  IDUS103408       F1A10_1                        7    Extreme Right   
1  IDUS103408       F1A10_1                        7    Extreme Right   
2  IDUS103408          F2A6                        4    Extreme Right   
3  IDUS103408          F2A6                        4    Extreme Right   
4  IDUS103408          F2A7                        1    Extreme Right   

   numeric_response_after   bias  response_change  
0                       5   left               -2  
1                       1  right               -6  
2                       4   left                0  
3                       5  right                1  
4                       5   left                4  


### Overall Changes in Responses Before and After Exposure to Articles

This step aims to assess how much the responses changed in general after exposure to articles. This involves calcultaing the average of the response_change across all agents and questions, and summarizing the overall shifts.

In [200]:

# Summary of overall changes in responses before and after exposure to articles
def overall_changes_analysis(merged_df):
    """
    Analyze the overall changes in responses before and after exposure to articles.
    """
    # Summary statistics of the response changes
    change_summary = merged_df['response_change'].describe()

    # Calculate the average response change
    avg_change = merged_df['response_change'].mean()

    # Count positive, negative, and no changes in response
    positive_changes = (merged_df['response_change'] > 0).sum()
    negative_changes = (merged_df['response_change'] < 0).sum()
    no_changes = (merged_df['response_change'] == 0).sum()

    return {
        'change_summary': change_summary,
        'avg_change': avg_change,
        'positive_changes': positive_changes,
        'negative_changes': negative_changes,
        'no_changes': no_changes
    }

# Perform the overall changes analysis
overall_change_results = overall_changes_analysis(merged_responses_df)

# Display the results
print("Overall Changes in Responses (Before vs After Exposure to Articles):")
for key, value in overall_change_results.items():
    print(f"{key}: {value}")


Overall Changes in Responses (Before vs After Exposure to Articles):
change_summary: count    1872.000000
mean       -0.162927
std         1.849973
min        -6.000000
25%        -1.000000
50%         0.000000
75%         0.000000
max         6.000000
Name: response_change, dtype: float64
avg_change: -0.16292735042735043
positive_changes: 423
negative_changes: 490
no_changes: 959


#### Overall difference in responses between left and right biased articles exposure


In [201]:
# Function to compare response changes between left- and right-biased articles
def compare_left_right_bias(merged_df):
    # Separate left-biased and right-biased responses
    left_bias_df = merged_df[merged_df['bias'] == 'left']
    right_bias_df = merged_df[merged_df['bias'] == 'right']

    # Calculate average response change for left and right bias
    avg_left_change = left_bias_df['response_change'].mean()
    avg_right_change = right_bias_df['response_change'].mean()

    # Calculate summary statistics for each bias
    left_summary = left_bias_df['response_change'].describe()
    right_summary = right_bias_df['response_change'].describe()

    # Count of positive and negative changes for each bias
    left_positive_changes = (left_bias_df['response_change'] > 0).sum()
    left_negative_changes = (left_bias_df['response_change'] < 0).sum()

    right_positive_changes = (right_bias_df['response_change'] > 0).sum()
    right_negative_changes = (right_bias_df['response_change'] < 0).sum()

    return {
        'avg_left_change': avg_left_change,
        'avg_right_change': avg_right_change,
        'left_summary': left_summary,
        'right_summary': right_summary,
        'left_positive_changes': left_positive_changes,
        'left_negative_changes': left_negative_changes,
        'right_positive_changes': right_positive_changes,
        'right_negative_changes': right_negative_changes
    }

# Perform the comparison between left and right bias
bias_comparison_results = compare_left_right_bias(merged_responses_df)

# Display the results
print("Comparison of Response Changes Between Left and Right Biased Articles:")
for key, value in bias_comparison_results.items():
    print(f"{key}: {value}")


Comparison of Response Changes Between Left and Right Biased Articles:
avg_left_change: -0.09188034188034189
avg_right_change: -0.23397435897435898
left_summary: count    936.000000
mean      -0.091880
std        1.728682
min       -6.000000
25%       -1.000000
50%        0.000000
75%        0.000000
max        6.000000
Name: response_change, dtype: float64
right_summary: count    936.000000
mean      -0.233974
std        1.962144
min       -6.000000
25%       -1.000000
50%        0.000000
75%        0.000000
max        6.000000
Name: response_change, dtype: float64
left_positive_changes: 217
left_negative_changes: 242
right_positive_changes: 206
right_negative_changes: 248


####   Overall differences in responses by question


In [202]:
# Function to analyze overall differences in responses by question
def analyze_differences_by_question(merged_df):
    # Group by question_code and calculate statistics for each question
    question_analysis = merged_df.groupby('question_code')['response_change'].agg(
        count='count',
        avg_change='mean',
        std_change='std',
        min_change='min',
        max_change='max',
        median_change='median'
    ).reset_index()

    # Sort the results by the average change to identify the most impacted questions
    question_analysis = question_analysis.sort_values(by='avg_change', ascending=False)

    return question_analysis

# Perform the analysis by question
question_analysis_results = analyze_differences_by_question(merged_responses_df)

# Display the results
print("Overall Differences in Responses by Question:")
print(question_analysis_results)


Overall Differences in Responses by Question:
  question_code  count  avg_change  std_change  min_change  max_change  \
2          F2A7    234    0.619658    1.845014          -4           4   
3          F2A9    234    0.064103    1.755480          -4           3   
7        F3A8_1    234    0.047009    1.320819          -6           6   
6        F3A7_1    234    0.008547    0.711593          -3           2   
5        F3A6_1    234   -0.085470    0.675413          -6           1   
1          F2A6    234   -0.175214    1.533344          -4           4   
4        F3A3_1    234   -0.341880    2.589720          -6           6   
0       F1A10_1    234   -1.440171    2.610364          -6           6   

   median_change  
2            0.0  
3            0.0  
7            0.0  
6            0.0  
5            0.0  
1            0.0  
4            0.0  
0            0.0  


#### Differences in responses by political groups

In [203]:
def analyze_differences_by_political_group(merged_df):
    # Group by political stance and calculate statistics for each group
    political_group_analysis = merged_df.groupby('political_stance')['response_change'].agg(
        count='count',
        avg_change='mean',
        std_change='std',
        min_change='min',
        max_change='max',
        median_change='median'
    ).reset_index()

    # Sort the results by the average change to identify the groups with the most impacted responses
    political_group_analysis = political_group_analysis.sort_values(by='avg_change', ascending=False)

    return political_group_analysis

# Perform the analysis by political group
political_group_analysis_results = analyze_differences_by_political_group(merged_responses_df)

# Display the results
print("Differences in Responses by Political Group:")
print(political_group_analysis_results)


Differences in Responses by Political Group:
  political_stance  count  avg_change  std_change  min_change  max_change  \
1    Extreme Right   1088   -0.034926    2.147434          -6           6   
0     Extreme Left    784   -0.340561    1.312125          -6           6   

   median_change  
1            0.0  
0            0.0  


#### Differences in responses by article bias: which bias causes the greater changes and in which direction?


In [204]:
def analyze_differences_by_article_bias_only(merged_df):
    # Group by article bias and calculate statistics for response changes
    bias_analysis = merged_df.groupby('bias')['response_change'].agg(
        count='count',
        avg_change='mean',
        std_change='std',
        min_change='min',
        max_change='max',
        median_change='median'
    ).reset_index()

    # Sort the results by the average change to easily compare bias impact
    bias_analysis = bias_analysis.sort_values(by='avg_change', ascending=False)

    return bias_analysis

# Perform the analysis by article bias only
bias_only_analysis_results = analyze_differences_by_article_bias_only(merged_responses_df)

# Display the results
print("Differences in Responses by Article Bias:")
print(bias_only_analysis_results)


Differences in Responses by Article Bias:
    bias  count  avg_change  std_change  min_change  max_change  median_change
0   left    936   -0.091880    1.728682          -6           6            0.0
1  right    936   -0.233974    1.962144          -6           6            0.0


#### Differences in responses by article bias by political group

In [205]:
# Function to analyze differences in responses by article bias and political group
def analyze_differences_by_article_bias_and_group(merged_df):
    # Group by article bias and political stance, then calculate statistics for response changes
    bias_group_analysis = merged_df.groupby(['bias', 'political_stance'])['response_change'].agg(
        count='count',
        avg_change='mean',
        std_change='std',
        min_change='min',
        max_change='max',
        median_change='median'
    ).reset_index()

    # Sort the results to easily compare bias impact
    bias_group_analysis = bias_group_analysis.sort_values(by='avg_change', ascending=False)

    return bias_group_analysis

# Perform the analysis by article bias and political group
bias_group_analysis_results = analyze_differences_by_article_bias_and_group(merged_responses_df)

# Display the results
print("Differences in Responses by Article Bias and Political Group:")
print(bias_group_analysis_results)


Differences in Responses by Article Bias and Political Group:
    bias political_stance  count  avg_change  std_change  min_change  \
1   left    Extreme Right    544    0.014706    1.978653          -6   
3  right    Extreme Right    544   -0.084559    2.304656          -6   
0   left     Extreme Left    392   -0.239796    1.292769          -6   
2  right     Extreme Left    392   -0.441327    1.325194          -6   

   max_change  median_change  
1           6            0.0  
3           6            0.0  
0           6            0.0  
2           5            0.0  


#### Differences in responses by article bias, by political group, and by question

In [206]:
# Function to analyze differences by article bias, political group, and question
def analyze_differences_by_bias_group_question(merged_df):
    # Group by article bias, political stance, and question_code, then calculate statistics for response changes
    bias_group_question_analysis = merged_df.groupby(['bias', 'political_stance', 'question_code'])['response_change'].agg(
        count='count',
        avg_change='mean',
        std_change='std',
        min_change='min',
        max_change='max',
        median_change='median'
    ).reset_index()

    # Sort the results by the average change to easily compare impact
    bias_group_question_analysis = bias_group_question_analysis.sort_values(by='avg_change', ascending=False)

    return bias_group_question_analysis

# Perform the analysis by article bias, political group, and question
bias_group_question_analysis_results = analyze_differences_by_bias_group_question(merged_responses_df)

# Display the results
print("Differences in Responses by Article Bias, Political Group, and Question:")
print(bias_group_question_analysis_results)


Differences in Responses by Article Bias, Political Group, and Question:
     bias political_stance question_code  count  avg_change  std_change  \
10   left    Extreme Right          F2A7     68    3.000000    0.646460   
27  right    Extreme Right          F2A9     68    2.073529    1.083336   
28  right    Extreme Right        F3A3_1     68    1.632353    2.278415   
15   left    Extreme Right        F3A8_1     68    0.691176    1.659503   
25  right    Extreme Right          F2A6     68    0.617647    1.051359   
6    left     Extreme Left        F3A7_1     49    0.408163    0.761533   
9    left    Extreme Right          F2A6     68    0.338235    1.140964   
0    left     Extreme Left       F1A10_1     49    0.244898    1.010994   
3    left     Extreme Left          F2A9     49    0.142857    0.500000   
16  right     Extreme Left       F1A10_1     49    0.102041    1.159111   
4    left     Extreme Left        F3A3_1     49    0.061224    1.375502   
29  right    Extreme Right 

### Stable Responses Analysis
Responses cannot get more extreme than what they already are, so they stay the same?

In [207]:
# Update the code to avoid the SettingWithCopyWarning
def check_stable_responses_at_extreme(merged_df, stable_threshold=0.5):
    # Filter for responses where response_change is close to zero (stable responses)
    stable_responses_df = merged_df[merged_df['response_change'].abs() <= stable_threshold].copy()
    
    # For F2 questions, the scale is 1-5, for others it's 1-7
    def is_at_extreme(row):
        if row['question_code'].startswith('F2'):
            return row['numeric_response_before'] in [1, 5]  # Extreme values for F2 questions
        else:
            return row['numeric_response_before'] in [1, 7]  # Extreme values for non-F2 questions
    
    # Apply the extreme check function to each row
    stable_responses_df.loc[:, 'at_extreme'] = stable_responses_df.apply(is_at_extreme, axis=1)
    
    # Calculate the number of stable responses that were at the extremes
    stable_at_extreme_count = stable_responses_df['at_extreme'].sum()
    
    # Calculate the percentage of stable responses that were at the extremes
    percentage_at_extreme = stable_at_extreme_count / len(stable_responses_df) * 100
    
    return {
        'total_stable_responses': len(stable_responses_df),
        'stable_at_extreme_count': stable_at_extreme_count,
        'percentage_at_extreme': percentage_at_extreme
    }

# Re-run the updated analysis to avoid the warning
extreme_stability_results = check_stable_responses_at_extreme(merged_responses_df)
percentage = extreme_stability_results["percentage_at_extreme"]
print(f"Percentage of stable responses at the extremes: {percentage:.2f}%")

Percentage of stable responses at the extremes: 82.59%


### Radicalization analysis:


#### Reinforcement of right wing opinions if right wing users are exposed to right wing biased articles

In [208]:
# Updated function to analyze reinforcement, stable extremes, and non-extreme responses not reinforced
def analyze_reinforcement_and_stable_extremes(merged_df):
    # Filter for Extreme Right users and right-wing biased articles
    right_wing_df = merged_df[(merged_df['political_stance'] == 'Extreme Right') & (merged_df['bias'] == 'right')].copy()

    # Function to check if the response was reinforced (became more extreme)
    def is_reinforced(row):
        if row['question_code'].startswith('F2'):  # F2 questions (concern scale)
            return row['numeric_response_after'] > row['numeric_response_before']  # More concern
        else:  # Agreement scale
            return (row['numeric_response_before'] < 4 and row['numeric_response_after'] < row['numeric_response_before']) or \
                   (row['numeric_response_before'] > 4 and row['numeric_response_after'] > row['numeric_response_before'])  # More extreme

    # Check if the non-reinforced response was already at an extreme (either 1 or 5 for F2, or 1 or 7 for others)
    def was_stable_at_extreme(row):
        if row['question_code'].startswith('F2'):
            return row['numeric_response_before'] in [1, 5]  # Extreme for F2 questions
        else:
            return row['numeric_response_before'] in [1, 7]  # Extreme for other questions

    # Apply the reinforcement check safely
    right_wing_df['reinforced'] = right_wing_df.apply(is_reinforced, axis=1)

    # Check if responses were stable at the extreme ends
    right_wing_df['stable_at_extreme'] = right_wing_df.apply(was_stable_at_extreme, axis=1)

    # Separate non-reinforced responses
    non_reinforced_df = right_wing_df[right_wing_df['reinforced'] == False].copy()

    # Check non-extreme responses and see how many were reinforced
    non_extreme_responses_df = right_wing_df[right_wing_df['stable_at_extreme'] == False].copy()
    non_extreme_reinforced_count = non_extreme_responses_df['reinforced'].sum()

    # Calculate percentages safely
    total_non_reinforced = len(non_reinforced_df)
    stable_at_extreme_count = non_reinforced_df['stable_at_extreme'].sum()

    # Prevent division by zero if no non-reinforced responses
    percentage_stable_at_extreme = (stable_at_extreme_count / total_non_reinforced * 100) if total_non_reinforced > 0 else 0

    total_non_extreme_responses = len(non_extreme_responses_df)
    percentage_non_extreme_reinforced = (non_extreme_reinforced_count / total_non_extreme_responses * 100) if total_non_extreme_responses > 0 else 0

    # Calculate non-extreme responses that were not reinforced
    non_extreme_not_reinforced_count = total_non_extreme_responses - non_extreme_reinforced_count
    percentage_non_extreme_not_reinforced = (non_extreme_not_reinforced_count / total_non_extreme_responses * 100) if total_non_extreme_responses > 0 else 0

    # Return results rounded to two decimal places
    return {
        'total_right_wing_responses': len(right_wing_df),
        'reinforced_responses': right_wing_df['reinforced'].sum(),
        'percentage_reinforced': round((right_wing_df['reinforced'].sum() / len(right_wing_df)) * 100, 2),
        'non_reinforced_responses': total_non_reinforced,
        'stable_at_extreme_count': stable_at_extreme_count,
        'percentage_stable_at_extreme': round(percentage_stable_at_extreme, 2),
        'non_extreme_reinforced_count': non_extreme_reinforced_count,
        'percentage_non_extreme_reinforced': round(percentage_non_extreme_reinforced, 2),
        'non_extreme_not_reinforced_count': non_extreme_not_reinforced_count,
        'percentage_non_extreme_not_reinforced': round(percentage_non_extreme_not_reinforced, 2)
    }

# Perform the extended radicalization analysis
right_wing_extended_results = analyze_reinforcement_and_stable_extremes(merged_responses_df)

# Display the results
print("Reinforcement of Right-Wing Opinions (Extreme Right Users, Right-Biased Articles):")
print(f"Total right-wing responses: {right_wing_extended_results['total_right_wing_responses']}")
print(f"Reinforced responses: {right_wing_extended_results['reinforced_responses']}")
print(f"Percentage of reinforced responses: {right_wing_extended_results['percentage_reinforced']}%")
print(f"Non-reinforced responses: {right_wing_extended_results['non_reinforced_responses']}")
print(f"Stable non-reinforced responses at extremes: {right_wing_extended_results['stable_at_extreme_count']}")
print(f"Percentage of stable non-reinforced responses at extremes: {right_wing_extended_results['percentage_stable_at_extreme']}%")
print(f"Non-extreme responses reinforced: {right_wing_extended_results['non_extreme_reinforced_count']}")
print(f"Percentage of non-extreme responses reinforced: {right_wing_extended_results['percentage_non_extreme_reinforced']}%")
print(f"Non-extreme responses not reinforced: {right_wing_extended_results['non_extreme_not_reinforced_count']}")
print(f"Percentage of non-extreme responses not reinforced: {right_wing_extended_results['percentage_non_extreme_not_reinforced']}%")


Reinforcement of Right-Wing Opinions (Extreme Right Users, Right-Biased Articles):
Total right-wing responses: 544
Reinforced responses: 115
Percentage of reinforced responses: 21.14%
Non-reinforced responses: 429
Stable non-reinforced responses at extremes: 312
Percentage of stable non-reinforced responses at extremes: 72.73%
Non-extreme responses reinforced: 102
Percentage of non-extreme responses reinforced: 46.58%
Non-extreme responses not reinforced: 117
Percentage of non-extreme responses not reinforced: 53.42%


#### Reinforcement of right wing opinions if left wing users are exposed to left wing biased articles

In [209]:
# Function to analyze reinforcement, stable extremes, and non-extreme responses not reinforced for left-wing users
def analyze_reinforcement_and_stable_extremes_left_wing(merged_df):
    # Filter for Extreme Left users and left-wing biased articles
    left_wing_df = merged_df[(merged_df['political_stance'] == 'Extreme Left') & (merged_df['bias'] == 'left')].copy()

    # Function to check if the response was reinforced (became more extreme)
    def is_reinforced(row):
        if row['question_code'].startswith('F2'):  # F2 questions (concern scale)
            return row['numeric_response_after'] > row['numeric_response_before']  # More concern
        else:  # Agreement scale
            return (row['numeric_response_before'] < 4 and row['numeric_response_after'] < row['numeric_response_before']) or \
                   (row['numeric_response_before'] > 4 and row['numeric_response_after'] > row['numeric_response_before'])  # More extreme

    # Check if the non-reinforced response was already at an extreme (either 1 or 5 for F2, or 1 or 7 for others)
    def was_stable_at_extreme(row):
        if row['question_code'].startswith('F2'):
            return row['numeric_response_before'] in [1, 5]  # Extreme for F2 questions
        else:
            return row['numeric_response_before'] in [1, 7]  # Extreme for other questions

    # Apply the reinforcement check safely
    left_wing_df['reinforced'] = left_wing_df.apply(is_reinforced, axis=1)

    # Check if responses were stable at the extreme ends
    left_wing_df['stable_at_extreme'] = left_wing_df.apply(was_stable_at_extreme, axis=1)

    # Separate non-reinforced responses
    non_reinforced_df = left_wing_df[left_wing_df['reinforced'] == False].copy()

    # Check non-extreme responses and see how many were reinforced
    non_extreme_responses_df = left_wing_df[left_wing_df['stable_at_extreme'] == False].copy()
    non_extreme_reinforced_count = non_extreme_responses_df['reinforced'].sum()

    # Calculate percentages safely
    total_non_reinforced = len(non_reinforced_df)
    stable_at_extreme_count = non_reinforced_df['stable_at_extreme'].sum()

    # Prevent division by zero if no non-reinforced responses
    percentage_stable_at_extreme = (stable_at_extreme_count / total_non_reinforced * 100) if total_non_reinforced > 0 else 0

    total_non_extreme_responses = len(non_extreme_responses_df)
    percentage_non_extreme_reinforced = (non_extreme_reinforced_count / total_non_extreme_responses * 100) if total_non_extreme_responses > 0 else 0

    # Calculate non-extreme responses that were not reinforced
    non_extreme_not_reinforced_count = total_non_extreme_responses - non_extreme_reinforced_count
    percentage_non_extreme_not_reinforced = (non_extreme_not_reinforced_count / total_non_extreme_responses * 100) if total_non_extreme_responses > 0 else 0

    # Return results rounded to two decimal places
    return {
        'total_left_wing_responses': len(left_wing_df),
        'reinforced_responses': left_wing_df['reinforced'].sum(),
        'percentage_reinforced': round((left_wing_df['reinforced'].sum() / len(left_wing_df)) * 100, 2),
        'non_reinforced_responses': total_non_reinforced,
        'stable_at_extreme_count': stable_at_extreme_count,
        'percentage_stable_at_extreme': round(percentage_stable_at_extreme, 2),
        'non_extreme_reinforced_count': non_extreme_reinforced_count,
        'percentage_non_extreme_reinforced': round(percentage_non_extreme_reinforced, 2),
        'non_extreme_not_reinforced_count': non_extreme_not_reinforced_count,
        'percentage_non_extreme_not_reinforced': round(percentage_non_extreme_not_reinforced, 2)
    }

# Perform the extended radicalization analysis for left-wing users
left_wing_extended_results = analyze_reinforcement_and_stable_extremes_left_wing(merged_responses_df)

# Display the results
print("Reinforcement of Left-Wing Opinions (Extreme Left Users, Left-Biased Articles):")
print(f"Total left-wing responses: {left_wing_extended_results['total_left_wing_responses']}")
print(f"Reinforced responses: {left_wing_extended_results['reinforced_responses']}")
print(f"Percentage of reinforced responses: {left_wing_extended_results['percentage_reinforced']}%")
print(f"Non-reinforced responses: {left_wing_extended_results['non_reinforced_responses']}")
print(f"Stable non-reinforced responses at extremes: {left_wing_extended_results['stable_at_extreme_count']}")
print(f"Percentage of stable non-reinforced responses at extremes: {left_wing_extended_results['percentage_stable_at_extreme']}%")
print(f"Non-extreme responses reinforced: {left_wing_extended_results['non_extreme_reinforced_count']}")
print(f"Percentage of non-extreme responses reinforced: {left_wing_extended_results['percentage_non_extreme_reinforced']}%")
print(f"Non-extreme responses not reinforced: {left_wing_extended_results['non_extreme_not_reinforced_count']}")
print(f"Percentage of non-extreme responses not reinforced: {left_wing_extended_results['percentage_non_extreme_not_reinforced']}%")


Reinforcement of Left-Wing Opinions (Extreme Left Users, Left-Biased Articles):
Total left-wing responses: 392
Reinforced responses: 79
Percentage of reinforced responses: 20.15%
Non-reinforced responses: 313
Stable non-reinforced responses at extremes: 231
Percentage of stable non-reinforced responses at extremes: 73.8%
Non-extreme responses reinforced: 79
Percentage of non-extreme responses reinforced: 49.07%
Non-extreme responses not reinforced: 82
Percentage of non-extreme responses not reinforced: 50.93%


### LLM to persona alignment before vs after exposure to articles: 
(is the llm more consistent with the actual person's responses after it was exposed to a. right biased article b. left biased articles.)

#### Data Preparation

In [210]:
import pandas as pd

# Load the filtered data (replace the path with the actual location of the CSV file)
filtered_data_path = '../data/processed/filtered_data.csv'
real_responses = pd.read_csv(filtered_data_path)



In [211]:
def weighted_difference(real_response_code, llm_response_code, weight=1):
    """
    Calculates the weighted difference between real and LLM response codes.
    
    Parameters:
    - real_response_code: The real response code (numeric or categorical).
    - llm_response_code: The response code generated by the LLM (numeric or categorical).
    - weight: A scaling factor for the difference. Defaults to 1 (no weighting).
    
    Returns:
    - The weighted difference between the real and LLM response codes.
    """
    # Ensure the codes are comparable by converting them to float (if they are numeric)
    try:
        real_response_code = float(real_response_code)
        llm_response_code = float(llm_response_code)
    except ValueError:
        # If not numeric, calculate difference as 1 if codes are different, 0 if the same
        return weight if real_response_code != llm_response_code else 0
    
    # Calculate the absolute difference between the codes, weighted by the provided factor
    difference = abs(real_response_code - llm_response_code)
    return difference * weight


In [212]:
def extract_llm_responses(after_responses_df):
    """
    Extracts LLM responses from after_responses_df and formats them into a list of dictionaries.
    Now includes 'bias' from the article.

    Returns:
    - llm_responses_mapped (list): A list of LLM response dictionaries, where each dictionary contains
                                   'user_id', 'question_code', 'llm_response_code', and 'bias'.
    """
    llm_responses_mapped = []
    
    # Extract index values from the multiindex in after_responses_df
    for (user_id, question_code, bias), row in after_responses_df.iterrows():
        llm_response_code = row['numeric_response']  # Assuming numeric_response contains the response
        
        # Append the response dictionary to the list, including the bias
        llm_responses_mapped.append({
            'user_id': user_id,
            'question_code': question_code,
            'llm_response_code': llm_response_code,
            'bias': bias
        })

    return llm_responses_mapped


In [213]:
def compare_llm_to_real(llm_responses_mapped, filtered_data):
    """
    Compare LLM responses to real responses and calculate the weighted differences.
    Now includes the 'bias' in the final DataFrame.

    Parameters:
    - llm_responses_mapped (list): A list of LLM response dictionaries, where each dictionary contains
                                   'user_id', 'question_code', 'llm_response_code', and 'bias'.
    - filtered_data (DataFrame): A pandas DataFrame containing real responses, with user_id as the index
                                 and question_code as the columns.

    Returns:
    - comparison_df (DataFrame): A pandas DataFrame containing the comparison results, including 'bias'.
    - average_weighted_difference (float): The average weighted difference between LLM and real responses.
    """
    llm_vs_real_comparison = []

    # Iterate through the LLM responses
    for llm_response in llm_responses_mapped:
        user_id = llm_response['user_id']
        question_code = llm_response['question_code']
        llm_response_code = llm_response['llm_response_code']
        bias = llm_response['bias']
        
        # Check if the user_id and question_code exist in filtered_data
        if user_id in filtered_data.index and question_code in filtered_data.columns:
            # Fetch the real response
            real_response = filtered_data.loc[user_id, question_code]
            
            # If real_response is a Series (multiple values), extract the first valid value
            if isinstance(real_response, pd.Series):
                real_response_code = real_response.iloc[0]  # Get the first value
            else:
                real_response_code = real_response
            
            # Check if the real response code is valid (not NaN)
            if pd.notna(real_response_code):
                # Calculate the weighted difference using the custom function
                difference = weighted_difference(real_response_code, llm_response_code)
                
                # Store the comparison
                llm_vs_real_comparison.append({
                    'user_id': user_id,
                    'question_code': question_code,
                    'llm_response_code': llm_response_code,
                    'real_response_code': real_response_code,
                    'difference': difference,
                    'bias': bias  # Include bias in the comparison DataFrame
                })

    # Convert the comparison results to a DataFrame for analysis
    comparison_df = pd.DataFrame(llm_vs_real_comparison)

    # Calculate the average weighted difference
    if not comparison_df.empty:
        average_weighted_difference = comparison_df['difference'].mean()
    else:
        average_weighted_difference = None  # Handle the case where there's no valid comparison

    return comparison_df, average_weighted_difference


In [214]:
# Set the unique_id as the index for real_responses_df
real_responses.set_index('unique_id', inplace=True)

# Assuming real_responses is the DataFrame containing the actual (real) responses from filtered_data.csv
comparison_df, avg_difference = compare_llm_to_real(llm_responses_mapped, real_responses)

# Display the comparison DataFrame
print(comparison_df.head())

# Display the average weighted difference
print(f"Average weighted difference between LLM and real responses: {avg_difference}")


      user_id question_code  llm_response_code  real_response_code  \
0  IDUS103408       F1A10_1                  5                   2   
1  IDUS103408       F1A10_1                  1                   2   
2  IDUS103408          F2A7                  5                   3   
3  IDUS103408          F2A7                  2                   3   
4  IDUS103408          F2A9                  2                   2   

   difference   bias  
0         3.0   left  
1         1.0  right  
2         2.0   left  
3         1.0  right  
4         0.0   left  
Average weighted difference between LLM and real responses: 1.9743589743589745


In [215]:
# Extract LLM responses from after_responses_df with bias
llm_responses_mapped = extract_llm_responses(after_responses_df)

# Compare LLM responses to real responses with bias tracking
comparison_df, avg_difference = compare_llm_to_real(llm_responses_mapped, real_responses)

# Display the results
print(comparison_df.head())
print(f"Average weighted difference between LLM and real responses: {avg_difference}")


      user_id question_code  llm_response_code  real_response_code  \
0  IDUS103408       F1A10_1                  5                   2   
1  IDUS103408       F1A10_1                  1                   2   
2  IDUS103408          F2A7                  5                   3   
3  IDUS103408          F2A7                  2                   3   
4  IDUS103408          F2A9                  2                   2   

   difference   bias  
0         3.0   left  
1         1.0  right  
2         2.0   left  
3         1.0  right  
4         0.0   left  
Average weighted difference between LLM and real responses: 1.9743589743589745


### Difference beteween LLM and Real Responses by Article Bias
Comparing the responses of the real people with the responses of their LLM counterparts that had been exposed to politically biased articles.

In [216]:
def analyze_responses_by_bias(comparison_df, bias_type):
    """
    Analyzes the comparison between real and LLM responses, filtered by bias.
    
    Parameters:
    - comparison_df (DataFrame): The DataFrame containing real and LLM response comparisons.
    - bias_type (str): The bias type to filter by ('right' or 'left'). Default is 'right'.
    
    Returns:
    - avg_difference (float): The average weighted difference for the selected bias type.
    - filtered_comparison_df (DataFrame): The filtered DataFrame containing only the selected bias type.
    """
    # Filter the DataFrame based on the selected bias
    filtered_comparison_df = comparison_df[comparison_df['bias'] == bias_type]
    
    # Calculate the average weighted difference for the selected bias
    avg_difference = filtered_comparison_df['difference'].mean()
    
    return avg_difference, filtered_comparison_df


In [217]:

# Example Usage:
# Analyze responses for right-biased articles
avg_difference_right, filtered_right_df = analyze_responses_by_bias(comparison_df, bias_type='right')

# Display the results
print("Comparison for Right-Biased Articles:")
print(filtered_right_df.head())
print(f"Average weighted difference for right-biased articles: {avg_difference_right}")

Comparison for Right-Biased Articles:
      user_id question_code  llm_response_code  real_response_code  \
1  IDUS103408       F1A10_1                  1                   2   
3  IDUS103408          F2A7                  2                   3   
5  IDUS103408          F2A9                  5                   2   
7  IDUS103408        F3A3_1                  5                   6   
9  IDUS103408        F3A6_1                  1                   4   

   difference   bias  
1         1.0  right  
3         1.0  right  
5         3.0  right  
7         1.0  right  
9         3.0  right  
Average weighted difference for right-biased articles: 1.9462759462759462


In [218]:

# Example Usage:
# Analyze responses for right-biased articles
avg_difference_right, filtered_right_df = analyze_responses_by_bias(comparison_df, bias_type='left')

# Display the results
print("Comparison for Left-Biased Articles:")
print(filtered_right_df.head())
print(f"Average weighted difference for left-biased articles: {avg_difference_right}")

Comparison for Left-Biased Articles:
      user_id question_code  llm_response_code  real_response_code  \
0  IDUS103408       F1A10_1                  5                   2   
2  IDUS103408          F2A7                  5                   3   
4  IDUS103408          F2A9                  2                   2   
6  IDUS103408        F3A3_1                  1                   6   
8  IDUS103408        F3A6_1                  2                   4   

   difference  bias  
0         3.0  left  
2         2.0  left  
4         0.0  left  
6         5.0  left  
8         2.0  left  
Average weighted difference for left-biased articles: 2.0024420024420024


#### Does Exposing left-wing mimicking LLMs to left biased articles improve their alignment with their real-human counterparts?

In [219]:
def add_political_stance_to_real_responses(real_responses_df, after_responses_df):
    """
    Adds the political stance to the real_responses_df by mapping from after_responses_df using 'user_id'.
    
    Parameters:
    - real_responses_df (DataFrame): The real responses DataFrame that lacks political stance.
    - after_responses_df (DataFrame): The after responses DataFrame that contains user_id and political_stance.
    
    Returns:
    - real_responses_df (DataFrame): The real responses DataFrame with added political stance.
    """
    # Reset index to bring 'user_id' and 'question_code' as columns in after_responses_df
    after_responses_df_reset = after_responses_df.reset_index()
    
    # Ensure after_responses_df has political stance and user_id
    stance_mapping = after_responses_df_reset[['user_id', 'political_stance']].drop_duplicates()
    
    # Merge the political stance into the real_responses_df using the index (which is unique_id)
    real_responses_with_stance = real_responses_df.merge(stance_mapping, left_index=True, right_on='user_id', how='left')
    
    return real_responses_with_stance


In [220]:
def track_reinforcement_patterns(df, political_stance, bias_type):
    """
    Track reinforcement patterns for users with a given political stance exposed to biased articles.
    
    Parameters:
    - df (DataFrame): The DataFrame containing the real and LLM responses, along with political stance and bias.
    - political_stance (str): The political stance to filter by (e.g., 'Extreme Left' or 'Extreme Right').
    - bias_type (str): The type of article bias to filter by (e.g., 'left' or 'right').

    Returns:
    - A dictionary with statistics about reinforcement patterns.
    """
    # Filter for the appropriate users and bias
    filtered_df = df[(df['political_stance'] == political_stance) & (df['bias'] == bias_type)].copy()

    # Determine reinforcement for each response
    filtered_df.loc[:, 'reinforced'] = filtered_df.apply(is_reinforced, axis=1)

    # Calculate statistics
    total_responses = len(filtered_df)
    reinforced_responses = filtered_df['reinforced'].sum()
    non_reinforced_responses = total_responses - reinforced_responses

    return {
        'total_responses': total_responses,
        'reinforced_responses': reinforced_responses,
        'percentage_reinforced': (reinforced_responses / total_responses * 100) if total_responses else 0,
        'non_reinforced_responses': non_reinforced_responses
    }


In [221]:
def compare_responses_for_left_users_and_left_bias(comparison_df, real_responses_with_stance):
    """
    Compares the responses of left-wing real people with their LLM counterparts after exposure to left-wing articles.
    
    Parameters:
    - comparison_df (DataFrame): The DataFrame containing real and LLM response comparisons.
    - real_responses_with_stance (DataFrame): The real responses DataFrame with political stance.
    
    Returns:
    - avg_difference (float): The average weighted difference for left-wing users exposed to left-biased articles.
    - filtered_comparison_df (DataFrame): The filtered DataFrame containing left-wing users and left-biased articles.
    """
    # Step 1: Ensure the correct index for real_responses_with_stance is set to 'user_id'
    real_responses_with_stance = real_responses_with_stance.set_index('user_id')
    
    # Step 2: Filter for left-wing users from real_responses_with_stance
    left_wing_users = real_responses_with_stance[real_responses_with_stance['political_stance'] == 'Extreme Left'].index.unique()
    
    print(f"Found {len(left_wing_users)} left-wing users:")

    # Step 3: Clean and ensure user ID matching (if necessary)
    comparison_df['user_id'] = comparison_df['user_id'].astype(str).str.strip()  # Ensure 'user_id' is a string and strip whitespace
    
    # Step 4: Filter the comparison_df for those users and left-biased articles
    filtered_comparison_df = comparison_df[
        (comparison_df['user_id'].isin(left_wing_users)) &
        (comparison_df['bias'] == 'left')
    ]
    
    print(f"Found {len(filtered_comparison_df)} entries with left bias for left-wing users.")


    # Step 5: Calculate the average weighted difference for left-wing users exposed to left-biased articles
    avg_difference = filtered_comparison_df['difference'].mean()
    
    return avg_difference, filtered_comparison_df






#### Left Wing Agents Right Articles - Human Alignment

In [222]:
def compare_responses_for_left_users_and_right_bias(comparison_df, real_responses_with_stance):
    """
    Compares the responses of left-wing real people with their LLM counterparts after exposure to right-wing articles.
    
    Parameters:
    - comparison_df (DataFrame): The DataFrame containing real and LLM response comparisons.
    - real_responses_with_stance (DataFrame): The real responses DataFrame with political stance.
    
    Returns:
    - avg_difference (float): The average weighted difference for left-wing users exposed to right-biased articles.
    - filtered_comparison_df (DataFrame): The filtered DataFrame containing left-wing users and right-biased articles.
    """
    # Step 1: Ensure the correct index for real_responses_with_stance is set to 'user_id'
    real_responses_with_stance = real_responses_with_stance.set_index('user_id')
    
    # Step 2: Filter for left-wing users from real_responses_with_stance
    left_wing_users = real_responses_with_stance[real_responses_with_stance['political_stance'] == 'Extreme Left'].index.unique()
    
    print(f"Found {len(left_wing_users)} left-wing users:")

    # Step 3: Clean and ensure user ID matching (if necessary)
    comparison_df['user_id'] = comparison_df['user_id'].astype(str).str.strip()  # Ensure 'user_id' is a string and strip whitespace
    
    # Step 4: Filter the comparison_df for those users and right-biased articles
    filtered_comparison_df = comparison_df[
        (comparison_df['user_id'].isin(left_wing_users)) &
        (comparison_df['bias'] == 'right')
    ]
    
    print(f"Found {len(filtered_comparison_df)} entries with right bias for left-wing users.")

    # Step 5: Calculate the average weighted difference for left-wing users exposed to right-biased articles
    avg_difference = filtered_comparison_df['difference'].mean()
    
    return avg_difference, filtered_comparison_df


#### Does Exposing right-wing mimicking LLMs to left biased articles improve their alignment with their real-human counterparts?

In [223]:
def compare_responses_for_right_users_and_right_bias(comparison_df, real_responses_with_stance):
    """
    Compares the responses of right-wing real people with their LLM counterparts after exposure to right-wing articles.
    
    Parameters:
    - comparison_df (DataFrame): The DataFrame containing real and LLM response comparisons.
    - real_responses_with_stance (DataFrame): The real responses DataFrame with political stance.
    
    Returns:
    - avg_difference (float): The average weighted difference for right-wing users exposed to right-biased articles.
    - filtered_comparison_df (DataFrame): The filtered DataFrame containing right-wing users and right-biased articles.
    """
    # Step 1: Ensure the correct index for real_responses_with_stance is set to 'user_id'
    real_responses_with_stance = real_responses_with_stance.set_index('user_id')
    
    # Step 2: Filter for right-wing users from real_responses_with_stance
    right_wing_users = real_responses_with_stance[real_responses_with_stance['political_stance'] == 'Extreme Right'].index.unique()
    
    print(f"Found {len(right_wing_users)} right-wing users:")

    # Step 3: Clean and ensure user ID matching (if necessary)
    comparison_df['user_id'] = comparison_df['user_id'].astype(str).str.strip()  # Ensure 'user_id' is a string and strip whitespace
    
    # Step 4: Filter the comparison_df for those users and right-biased articles
    filtered_comparison_df = comparison_df[
        (comparison_df['user_id'].isin(right_wing_users)) &
        (comparison_df['bias'] == 'right')
    ]
    
    print(f"Found {len(filtered_comparison_df)} entries with right bias for right-wing users.")

    # Step 5: Calculate the average weighted difference for right-wing users exposed to right-biased articles
    avg_difference = filtered_comparison_df['difference'].mean()
    
    return avg_difference, filtered_comparison_df


In [229]:
def display_comparison_results(comparison_df, real_responses, after_responses_df):
    """
    Compares and displays results for left-wing and right-wing users exposed to both left- and right-biased articles.
    
    Parameters:
    - comparison_df (DataFrame): The DataFrame containing real and LLM response comparisons.
    - real_responses (DataFrame): The real responses DataFrame that lacks political stance.
    - after_responses_df (DataFrame): The after responses DataFrame that contains user_id and political_stance.
    """
    # Step 1: Add political stance to real_responses
    real_responses_with_stance = add_political_stance_to_real_responses(real_responses, after_responses_df)

    # ---- Right-Wing Users ----
    print("\n--- Right-Wing Users Comparisons ---\n")

    # Compare responses of right-wing users with right-biased articles
    avg_difference_right, filtered_right_df = compare_responses_for_right_users_and_right_bias(comparison_df, real_responses_with_stance)
    print("Comparison for Right-Wing Users Exposed to Right-Biased Articles:")
    print(f"Average weighted difference for right-wing users and right-biased articles: {avg_difference_right}")
    print()

    # Compare responses of right-wing users with left-biased articles
    avg_difference_right_left, filtered_right_left_df = compare_responses_for_right_users_and_left_bias(comparison_df, real_responses_with_stance)
    print("Comparison for Right-Wing Users Exposed to Left-Biased Articles:")
    print(f"Average weighted difference for right-wing users and left-biased articles: {avg_difference_right_left}")
    print()

    # ---- Left-Wing Users ----
    print("\n--- Left-Wing Users Comparisons ---\n")

    # Compare responses of left-wing users with left-biased articles
    avg_difference_left, filtered_left_df = compare_responses_for_left_users_and_left_bias(comparison_df, real_responses_with_stance)
    print("Comparison for Left-Wing Users Exposed to Left-Biased Articles:")
    print(f"Average weighted difference for left-wing users and left-biased articles: {avg_difference_left}")
    print()

    # Compare responses of left-wing users with right-biased articles
    avg_difference_left_right, filtered_left_right_df = compare_responses_for_left_users_and_right_bias(comparison_df, real_responses_with_stance)
    print("Comparison for Left-Wing Users Exposed to Right-Biased Articles:")
    print(f"Average weighted difference for left-wing users and right-biased articles: {avg_difference_left_right}")
    print()


In [230]:
display_comparison_results(comparison_df, real_responses, after_responses_df)



--- Right-Wing Users Comparisons ---

Found 68 right-wing users:
Found 476 entries with right bias for right-wing users.
Comparison for Right-Wing Users Exposed to Right-Biased Articles:
Average weighted difference for right-wing users and right-biased articles: 1.9453781512605042

Found 68 right-wing users:
Found 476 entries with left bias for right-wing users.
Comparison for Right-Wing Users Exposed to Left-Biased Articles:
Average weighted difference for right-wing users and left-biased articles: 2.0105042016806722


--- Left-Wing Users Comparisons ---

Found 49 left-wing users:
Found 343 entries with left bias for left-wing users.
Comparison for Left-Wing Users Exposed to Left-Biased Articles:
Average weighted difference for left-wing users and left-biased articles: 1.9912536443148687

Found 49 left-wing users:
Found 343 entries with right bias for left-wing users.
Comparison for Left-Wing Users Exposed to Right-Biased Articles:
Average weighted difference for left-wing users and 

### Right Wing Users Exposed to Left-Biased Articles Human Alignment

In [225]:
def compare_responses_for_right_users_and_left_bias(comparison_df, real_responses_with_stance):
    """
    Compares the responses of right-wing real people with their LLM counterparts after exposure to left-wing articles.
    
    Parameters:
    - comparison_df (DataFrame): The DataFrame containing real and LLM response comparisons.
    - real_responses_with_stance (DataFrame): The real responses DataFrame with political stance.
    
    Returns:
    - avg_difference (float): The average weighted difference for right-wing users exposed to left-biased articles.
    - filtered_comparison_df (DataFrame): The filtered DataFrame containing right-wing users and left-biased articles.
    """
    # Step 1: Ensure the correct index for real_responses_with_stance is set to 'user_id'
    real_responses_with_stance = real_responses_with_stance.set_index('user_id')
    
    # Step 2: Filter for right-wing users from real_responses_with_stance
    right_wing_users = real_responses_with_stance[real_responses_with_stance['political_stance'] == 'Extreme Right'].index.unique()
    
    print(f"Found {len(right_wing_users)} right-wing users:")

    # Step 3: Clean and ensure user ID matching (if necessary)
    comparison_df['user_id'] = comparison_df['user_id'].astype(str).str.strip()  # Ensure 'user_id' is a string and strip whitespace
    
    # Step 4: Filter the comparison_df for those users and left-biased articles
    filtered_comparison_df = comparison_df[
        (comparison_df['user_id'].isin(right_wing_users)) &
        (comparison_df['bias'] == 'left')
    ]
    
    print(f"Found {len(filtered_comparison_df)} entries with left bias for right-wing users.")

    # Step 5: Calculate the average weighted difference for right-wing users exposed to left-biased articles
    avg_difference = filtered_comparison_df['difference'].mean()
    
    return avg_difference, filtered_comparison_df


### Radicalisation LLMs and Humans 
Instead of comparing LLMs to LLMs for radicalisation analysis, as done before, we now compare LLMs and humans.

In [226]:
def is_reinforced(row):
    """
    Determines if the LLM response is a reinforcement of the user's original stance.
    Reinforcement happens if the LLM response moves in the direction of the original political stance.
    
    For Extreme Left:
      - Moving towards smaller numbers (more extreme left) is considered reinforcement.
    For Extreme Right:
      - Moving towards larger numbers (more extreme right) is considered reinforcement.
    
    Returns True if reinforcement is detected, False otherwise.
    """
    if row['political_stance'] == 'Extreme Left' and row['bias'] == 'left':
        return row['llm_response_code'] < row['real_response_code']  # moving towards more extreme left
    elif row['political_stance'] == 'Extreme Right' and row['bias'] == 'right':
        return row['llm_response_code'] > row['real_response_code']  # moving towards more extreme right
    return False  # No reinforcement otherwise


In [227]:
def track_reinforcement_patterns(df, political_stance, bias_type):
    """
    Track reinforcement patterns for users with a given political stance exposed to biased articles.
    
    Parameters:
    - df (DataFrame): The DataFrame containing the real and LLM responses, along with political stance and bias.
    - political_stance (str): The political stance to filter by (e.g., 'Extreme Left' or 'Extreme Right').
    - bias_type (str): The type of article bias to filter by (e.g., 'left' or 'right').

    Returns:
    - A dictionary with statistics about reinforcement patterns.
    """
    # Filter for the appropriate users and bias
    filtered_df = df[(df['political_stance'] == political_stance) & (df['bias'] == bias_type)].copy()

    # Determine reinforcement for each response
    filtered_df.loc[:, 'reinforced'] = filtered_df.apply(is_reinforced, axis=1)

    # Calculate statistics
    total_responses = len(filtered_df)
    reinforced_responses = filtered_df['reinforced'].sum()
    non_reinforced_responses = total_responses - reinforced_responses

    return {
        'total_responses': total_responses,
        'reinforced_responses': reinforced_responses,
        'percentage_reinforced': (reinforced_responses / total_responses * 100) if total_responses else 0,
        'non_reinforced_responses': non_reinforced_responses
    }


In [228]:
# Ensure columns are correctly named in the DataFrame
merged_responses_df.rename(columns={
    'numeric_response_after': 'llm_response_code',
    'numeric_response_before': 'real_response_code'
}, inplace=True)

# Example usage: Track reinforcement patterns for left-wing users exposed to left-biased articles
reinforcement_stats_left = track_reinforcement_patterns(merged_responses_df, 'Extreme Left', 'left')
reinforcement_stats_right = track_reinforcement_patterns(merged_responses_df, 'Extreme Right', 'right')

# Output the reinforcement stats
print("Reinforcement patterns for left-wing users exposed to left-biased articles:")
print(reinforcement_stats_left)

print("Reinforcement patterns for right-wing users exposed to right-biased articles:")
print(reinforcement_stats_right)


Reinforcement patterns for left-wing users exposed to left-biased articles:
{'total_responses': 392, 'reinforced_responses': np.int64(109), 'percentage_reinforced': np.float64(27.806122448979593), 'non_reinforced_responses': np.int64(283)}
Reinforcement patterns for right-wing users exposed to right-biased articles:
{'total_responses': 544, 'reinforced_responses': np.int64(148), 'percentage_reinforced': np.float64(27.205882352941174), 'non_reinforced_responses': np.int64(396)}


In [ ]:
# TODO: Do llms show similar patterns of radicalisation as humans? 
# TODO: Check alignment with humans question by question after article exposure.